In [1]:
"""
Downloaded all 20 of the 2014-2015 region files from 
http://ritter.tea.state.tx.us/adhocrpt/Disciplinary_Data_Products/Download_Region_Districts.html 
and pasted them together (as TX2015.csv).

Also downloaded "District and Charter Detail Data" (Snapshot 2015) from 
http://ritter.tea.state.tx.us/perfreport/snapshot/download.html (as district2015.csv)

"""

import pandas as pd

year = "2016" # put the year to use to label the output file (for 2015-16, I use "2016")
apple = pd.read_csv('TX2016.csv') # put the name of the discipline actions file here
district = pd.read_csv('district2016.csv') # put the name of the district demographics file here

# apple[:5]

In [2]:
# Delete rows that repeat the headers.

apple = apple[apple["REGION"].str.contains("REGION") == False]


In [3]:
# Changing the reference to 2015 in the column name might help prevent bugs when adapting this file for later years.

import re

apple = apple.rename(columns=lambda x: re.sub("YR\d\d","Group Punishments",x))

In [4]:
apple[:5]

,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,Group Punishments
0,DISTRICT SUMMARY,01,BROWNSVILLE ISD,031901,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,50150
1,DISTRICT SUMMARY,01,BROWNSVILLE ISD,031901,A-PARTICIPATION,A02,DISTRICT DISCIPLINE POPULATION,4127
2,DISTRICT SUMMARY,01,BROWNSVILLE ISD,031901,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,12158
3,DISTRICT SUMMARY,01,BROWNSVILLE ISD,031901,B-DISCIPLINE DATA TRENDS,B01,COUNT OF STUDENTS EXPELLED TO JJAEP,9
4,DISTRICT SUMMARY,01,BROWNSVILLE ISD,031901,B-DISCIPLINE DATA TRENDS,B02,MANDATORY EXPULSIONS TO JJAEP,-99999999


In [5]:
# deleting redundant columns

apple = apple[['DISTRICT','SECTION', 'HEADING NAME', "Group Punishments"]]
district = district[['DISTRICT','DISTNAME','REGION','DPETALLC','DPETBLAP','DPETHISP','DPETWHIP','DPETINDP','DPETASIP',
                    'DPETPCIP','DPETTWOP','DPETECOP','DPETSPEP']]

In [6]:
# string columns to int

apple = apple.astype({'DISTRICT': int, "Group Punishments": int})

apple[:20]

,DISTRICT,SECTION,HEADING NAME,Group Punishments
0,31901,A-PARTICIPATION,DISTRICT CUMULATIVE YEAR END ENROLLMENT,50150
1,31901,A-PARTICIPATION,DISTRICT DISCIPLINE POPULATION,4127
2,31901,A-PARTICIPATION,DISTRICT DISCIPLINE RECORD COUNT,12158
3,31901,B-DISCIPLINE DATA TRENDS,COUNT OF STUDENTS EXPELLED TO JJAEP,9
4,31901,B-DISCIPLINE DATA TRENDS,MANDATORY EXPULSIONS TO JJAEP,-99999999
5,31901,B-DISCIPLINE DATA TRENDS,DISCRETIONARY EXPULSIONS TO JJAEP,-99999999
6,31901,B-DISCIPLINE DATA TRENDS,COUNT OF STUDENTS EXPELLED,10
7,31901,B-DISCIPLINE DATA TRENDS,MANDATORY EXPULSIONS,-99999999
8,31901,B-DISCIPLINE DATA TRENDS,DISCRETIONARY EXPULSIONS,-99999999
9,31901,B-DISCIPLINE DATA TRENDS,COUNT OF STUDENTS REMOVED TO A DAEP,541


In [7]:
# Keeping only the rows that categorize students by protected class. 
# Also getting rid of rows that count students instead of incidents, or non-disadvantaged kids.

patternIn = 'WHITE|BLACK OR AFRICAN AMERICAN|AMERICAN INDIAN OR ALASKA NAT|HISPANIC|NATIVE HAWAIIAN|ASIAN|TWO OR MORE RACES|SPEC. ED|ECO. DISAD|ECO DISAD.'
patternOut = 'SPEC. ED. STUDENTS| SPEC. ED. EXPULSIONS TO JJAEP|ECO DISAD. STUDENTS|ECO. DISAD. STUDENTS|AT RISK|NON AT|UNKNOWN AT|NON SPEC. ED.|NON ECO DISAD.|NON ECO. DISAD.'

apple = apple[apple["HEADING NAME"].str.contains(patternIn)]
apple = apple[apple["HEADING NAME"].str.contains(patternOut) == False]

# Delete rows appearing to double-count the same expulsions.

apple = apple[apple["SECTION"].str.contains("JJAEP EXPULSIONS|DISCIPLINE ACTION COUNTS") == False]

In [8]:
# Consolidating some of the descriptors into broader categories

appleReplace = {"Group Punishments":
                    {-99999999: 1}, 
                "SECTION":{
            'M-ECO\. DISADV\. JJAEP PLACEMENTS|H-SPEC\. ED\. JJAEP EXPULSIONS':'C-JJAEP EXPULSIONS',
            'N-ECO\. DISADV\. EXPULSIONS|I-SPEC\. ED\. EXPULSIONS': 'D-EXPULSION ACTIONS',
            'O-ECO\. DISADV\. DAEP PLACEMENTS|J-SPEC\. ED\. DAEP PLACEMENTS': 'E-DAEP PLACEMENTS',
            'P-ECO\. DISADV\. OUT OF SCHOOL SUS.|K-SPEC\. ED\. OUT OF SCHOOL SUS\.': 'F-OUT OF SCHOOL SUSPENSIONS',
            'Q-ECO\. DISADV\. IN SCHOOL SUS\.|L-SPEC\. ED\. IN SCHOOL SUS\.': 'G-IN SCHOOL SUSPENSIONS'},
                "HEADING NAME": {'SPEC\. ED.*$': 'Special Education',
                                 'ECO?. DISAD.*$': 'Economic Disadvantage'}
        }

apple = apple.replace(to_replace = appleReplace, regex = True)

In [9]:
groups = {'AMERICAN INDIAN OR ALASKA NAT': 'DPETINDP',
 'ASIAN': 'DPETASIP',
 'BLACK OR AFRICAN AMERICAN': 'DPETBLAP',
 'Economic Disadvantage': 'DPETECOP',
 'HISPANIC/LATINO': 'DPETHISP',
 'NATIVE HAWAIIAN/OTHER PACIFIC': 'DPETPCIP',
 'Special Education': 'DPETSPEP',
 'TWO OR MORE RACES': 'DPETTWOP',
 'WHITE': 'DPETWHIP'}

# Using percentages to get counts of students in each group.

for key in groups:
    district[key] = district["DPETALLC"] * district[groups[key]] // 100
    
district = district.astype({'BLACK OR AFRICAN AMERICAN':int, 'HISPANIC/LATINO': int, 'WHITE': int, 
          'AMERICAN INDIAN OR ALASKA NAT': int, 'ASIAN': int, 'NATIVE HAWAIIAN/OTHER PACIFIC': int,
         'TWO OR MORE RACES': int, 'Economic Disadvantage': int, 'Special Education': int})

# district[:5]

In [10]:
distPop = district[['DISTRICT','DPETALLC']]

# distPop[:5]

In [11]:
appleRace = apple[apple["HEADING NAME"].str.contains("Economic Disadvantage|Special Education") == False]

appleRace[10:25]

,DISTRICT,SECTION,HEADING NAME,Group Punishments
25,31901,G-IN SCHOOL SUSPENSIONS,WHITE,41
112,108902,D-EXPULSION ACTIONS,HISPANIC/LATINO,11
113,108902,E-DAEP PLACEMENTS,HISPANIC/LATINO,475
114,108902,F-OUT OF SCHOOL SUSPENSIONS,HISPANIC/LATINO,3633
115,108902,F-OUT OF SCHOOL SUSPENSIONS,WHITE,5
116,108902,G-IN SCHOOL SUSPENSIONS,HISPANIC/LATINO,7170
117,108902,G-IN SCHOOL SUSPENSIONS,TWO OR MORE RACES,1
118,108902,G-IN SCHOOL SUSPENSIONS,WHITE,1
206,108903,E-DAEP PLACEMENTS,BLACK OR AFRICAN AMERICAN,1
207,108903,E-DAEP PLACEMENTS,HISPANIC/LATINO,1


In [12]:
appleRace = appleRace.set_index(["DISTRICT","SECTION"])
appleRace["PUNISHMENTS"] = appleRace["Group Punishments"].groupby(level = ['DISTRICT','SECTION']).sum()

# appleRace[10:25]

In [13]:
applePunishments = appleRace.reset_index()
applePunishments = applePunishments[["DISTRICT","SECTION","PUNISHMENTS"]]

applePunishments = applePunishments.drop_duplicates()
applePunishments[10:25]

,DISTRICT,SECTION,PUNISHMENTS
24,108903,G-IN SCHOOL SUSPENSIONS,1811
28,108904,D-EXPULSION ACTIONS,8
29,108904,E-DAEP PLACEMENTS,900
34,108904,F-OUT OF SCHOOL SUSPENSIONS,6847
39,108904,G-IN SCHOOL SUSPENSIONS,14035
44,108809,F-OUT OF SCHOOL SUSPENSIONS,1
45,31903,D-EXPULSION ACTIONS,28
46,31903,E-DAEP PLACEMENTS,637
50,31903,F-OUT OF SCHOOL SUSPENSIONS,916
54,31903,G-IN SCHOOL SUSPENSIONS,4393


In [14]:
apple = pd.merge(apple, applePunishments, on=['DISTRICT', 'SECTION'])

# apple[10:25]

In [15]:
apple = pd.merge(apple, distPop, how='outer', on=['DISTRICT'])

# apple[10:25]

In [16]:
district = district.set_index("DISTRICT")

In [17]:
def getRacePop(df, row):
    return df.ix[row["DISTRICT"]][row["HEADING NAME"]]

# getRacePop(district, apple[:3])

In [18]:
apple = apple.dropna() # disregarding lines with null values. They were creating errors.

In [19]:
# Temporarily moving this information from the district dataframe to the punishment dataframe to make later
# calculations easier.

apple["DEMO POPULATION"] = apple.apply(lambda x: getRacePop(district, x), axis = 1)

# apple[10:25]

In [20]:
# Calculating the ratio of punishments for the demographic group compared to the punishments for the student population
# as a whole. For instance, "0.505" in the disparity column indicates the group got the punishment 50.5% as often
# as average for the student population.

from scipy import stats
from decimal import Decimal

def getRatio(distPop, racePop, all_punishments, group_punishments):
    if max(racePop, group_punishments) == 0 or None:
        return None
    elif all_punishments == 0 or None:
        return 0
    else:
        disparity = (group_punishments / (max(all_punishments, group_punishments)) \
                     / (max(racePop, group_punishments) / distPop)) - 1
        disparity = Decimal(disparity)
        disparity = disparity.quantize(Decimal('0.01'))
    return float(disparity)

apple["Disparity"] = apple.apply(lambda x: getRatio(x["DPETALLC"],x["DEMO POPULATION"], x["PUNISHMENTS"], \
                              x["Group Punishments"]), axis=1)

# apple[:10]

In [21]:
# The "RecordError" column flags implausible data entries. Some of them could still be true if school administrators
# applied different standards different standards to determine which students belong to which demographic group.
# Or some could be the result of students not being counted because of the time they moved in and out of the district.

def impossible(distPop, racePop, all_punishments, group_punishments):
    impossible = False
    if group_punishments > all_punishments or racePop > distPop:
        impossible = True
    if racePop == 0 and group_punishments > 0:
        impossible = True
    return impossible

apple["LikelyError"] = apple.apply(lambda x: impossible(x["DPETALLC"],x["DEMO POPULATION"], x["PUNISHMENTS"], \
                              x["Group Punishments"]), axis=1)

In [22]:
apple = apple.astype({'Group Punishments':int})

# apple[:10]

In [23]:
# I don't know if this is a valid way to report the Fisher's exact test statistic, but the idea is that if getFisher returns a 
# positive number over .95, there's a 95% chance that the group's better-than-average treatment is not due to chance.
# If it returns a number under -.95, there's a 95% chance that the group's worse-than-average treatment is not due to chance.
# I think it should be easier to create a color scale to show the scores on a map this way.

# The getFisher function assumes wrongly that everyone can have only one punishment (of each type). If the number of 
# punishments exceeds the number of kids, it reduces the number of punishments (and assumes wrongly that every 
# kid has been punished) But maybe the results are still close enough to correct to use for scaling?

import numpy as np

def getFisher(distPop, racePop, all_punishments, group_punishments):
    if max(racePop, group_punishments) == 0 or None:
        return None, None
    elif all_punishments == 0 or None:
        return 1, 0
    else:
        oddsratio, pvalueG = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                [group_punishments, max(all_punishments - group_punishments, 0)]], 
                                               alternative = 'greater')
        oddsratio, pvalueL = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                [group_punishments, max(all_punishments - group_punishments, 0)]], 
                                               alternative = 'less')
        if pvalueL < pvalueG:
            pv = 1 - pvalueL
        else:
            pv = pvalueG - 1
        pv = Decimal(pv)
        pv = pv.quantize(Decimal('0.000001'))
    return float(pv)

apple["Scale"] = apple.apply(lambda x: getFisher(x["DPETALLC"],x["DEMO POPULATION"], x["PUNISHMENTS"], \
                              x["Group Punishments"]), axis=1)

In [24]:
district[:3]

,DISTNAME,REGION,DPETALLC,DPETBLAP,DPETHISP,DPETWHIP,DPETINDP,DPETASIP,DPETPCIP,DPETTWOP,...,DPETSPEP,WHITE,Special Education,TWO OR MORE RACES,BLACK OR AFRICAN AMERICAN,AMERICAN INDIAN OR ALASKA NAT,Economic Disadvantage,ASIAN,HISPANIC/LATINO,NATIVE HAWAIIAN/OTHER PACIFIC
DISTRICT,,,,,,,,,,,,,,,,,,,,,
1902,CAYUGA ISD,7,568,4.2,7.0,82.9,0.0,0.7,0.0,5.1,...,13.7,470,77,28,23,0,191,3,39,0
1903,ELKHART ISD,7,1244,5.4,9.6,81.4,0.1,0.9,0.2,2.6,...,11.6,1012,144,32,67,1,680,11,119,2
1904,FRANKSTON ISD,7,841,8.8,10.2,75.9,0.5,0.7,0.1,3.8,...,8.2,638,68,31,74,4,475,5,85,0


In [25]:
apple = apple[['DISTRICT','SECTION', 'HEADING NAME', "Group Punishments", "Disparity", "Scale", "LikelyError"]]
district.reset_index(level=0, inplace=True)
district = district[['DISTRICT','DISTNAME','DPETALLC','ASIAN','AMERICAN INDIAN OR ALASKA NAT','NATIVE HAWAIIAN/OTHER PACIFIC',\
                     'HISPANIC/LATINO','BLACK OR AFRICAN AMERICAN','TWO OR MORE RACES','Special Education','WHITE',\
                     'Economic Disadvantage']]

In [26]:
DisparitiesFile = "DistrictDisparities" + str(year) + ".csv"
DemoFile = "TXdemo" + str(year) + ".csv"

In [27]:
apple.to_csv(path_or_buf = DisparitiesFile, columns = ['DISTRICT','SECTION', 'HEADING NAME', \
                                        "Group Punishments", "Disparity", "Scale", "LikelyError"], index = False)

district.to_csv(path_or_buf = DemoFile, index = False)

In [30]:
# apple["LikelyError"].describe()

count     17141
unique        2
top       False
freq      16550
Name: LikelyError, dtype: object